In [2]:
import csv
import pandas as pd
from IPython.display import display

In [3]:
with open('toxicity_annotations_unanimous.tsv','r') as tsv_toxicity_annotations:
    toxicity_annotations = [line.strip().split('\t') for line in tsv_toxicity_annotations]

with open('toxicity_annotated_comments_unanimous.tsv','r') as tsv_toxicity_comments:
    toxicity_annotations_comments = [line.strip().split('\t') for line in tsv_toxicity_comments]

In [4]:
toxicity_annotations_df = pd.read_table('toxicity_annotations_unanimous.tsv')
toxicity_annotations_comments_df = pd.read_table('toxicity_annotated_comments_unanimous.tsv')

In [ ]:
#Comput mean and median scores from annotations dataset
toxicity_annotations_comments_df["mean_score"] = pd.Series(toxicity_annotations_df.groupby("rev_id",as_index=False).mean()["toxicity_score"])
toxicity_annotations_comments_df["median_score"] = pd.Series(toxicity_annotations_df.groupby("rev_id",as_index=False).median()["toxicity_score"])
# create binary variable toxicity: if median score < 0, toxicity=1, otherwise 0
toxicity_annotations_comments_df["toxicity"] = (toxicity_annotations_comments_df["median_score"] < 0).astype(int)

#display (toxicity_annotations_comments_df)

In [ ]:
#Replace "newline token" with \n
toxicity_annotations_comments_df['comment'].replace('NEWLINE_TOKEN',r'\n',regex=True,inplace=True)

#Compute comment length for each comment and store it as a table
toxicity_annotations_comments_df['comment_len'] = toxicity_annotations_comments_df['comment'].apply(len)

In [137]:
#group data based off rev_id and score
toxicity_grouped = toxicity_annotations_comments_df.groupby(['rev_id' , 'median_score'])

#intialize empty dict
toxicity_dict = {"-2.0": 0, "-1.0": 0, "0.0": 0, "1.0": 0, "2.0": 0}
comment_length_dict = {"-2.0": 0, "-1.0": 0, "0.0": 0, "1.0": 0, "2.0": 0}

#build a histogram dict
for name, group in toxicity_grouped:
    rev_id,toxicity = name
    toxicity_dict[str(toxicity)] = toxicity_dict[str(toxicity)] + 1 
    
print ("toxicity hist is" + str(toxicity_dict))

for median in range(-2,2,1):
    comment_length_dict[str(float(median))] = (toxicity_annotations_comments_df.loc[toxicity_annotations_comments_df['median_score']==float(median)]['comment_len']).mean()    
print ("comment length hist is " + str(comment_length_dict))

toxicity hist is{'-2.0': 12, '-1.0': 159, '0.0': 3187, '1.0': 224, '2.0': 0}
comment length hist is {'-2.0': 2163.0, '-1.0': 187.77987421383648, '0.0': 91.54910574207719, '1.0': 453.9732142857143, '2.0': 0}


In [ ]:
#Merge the two panda tables
#toxicity_annotations_combined = pd.merge(toxicity_annotations_comments_df,toxicity_annotations_df,on='rev_id')
#display (toxicity_annotations_comments_df)